In [1]:
import pandas as pd
import QUANTAXIS as qtx
from datetime import datetime, timedelta

In [2]:
TOP_STOCKS=500
CHECK_DAYS=30

In [3]:
def mtm_ind(data, col, N=12, M=6):
    mtm = (data[col] - qtx.REF(data[col], N)) / qtx.REF(data[col], N)
    mtmma = qtx.MA(mtm, M)
    return pd.DataFrame({'MTM': mtm, 'MTMMA': mtmma})

In [4]:
code_list = qtx.QA_fetch_stock_list()
finfo =  qtx.QA_fetch_stock_info(list(code_list.code))
fcnd = qtx.QA_fetch_stock_day_adv(list(code_list.code),
                                  start=(datetime.today()-timedelta(days=CHECK_DAYS+30)).strftime('%Y-%m-%d'), 
                                  end=datetime.today().strftime('%Y-%m-%d')).to_qfq()


In [5]:
fbase = fcnd.data.join(finfo.loc[finfo.ipo_date != 0, ['ipo_date','industry']], how='inner').reset_index()
fbase['ipodays'] = fbase.apply(lambda x: (x.date - datetime.strptime(str(x.ipo_date), '%Y%m%d')).days, axis=1)
fbase = fbase.set_index(['date', 'code']) 
fbase.head()

open   high    low  close    volume        amount  adj  \
date       code                                                              
2020-09-14 000001  15.01  15.30  14.92  15.30  680251.0  1.027508e+09  1.0   
           000002  28.01  28.30  27.51  27.70  588402.0  1.630472e+09  1.0   
           000004  22.99  23.35  22.91  23.18   11770.0  2.722349e+07  1.0   
           000005   2.88   2.90   2.84   2.86   89083.0  2.551092e+07  1.0   
           000006   5.87   5.88   5.70   5.80  171019.0  9.886574e+07  1.0   

                   ipo_date  industry  ipodays  
date       code                                 
2020-09-14 000001  19910403         1    10757  
           000002  19910129        11    10821  
           000004  19910114        24    10836  
           000005  19901210        39    10871  
           000006  19920427        11    10367

In [6]:
mtm=fcnd.add_func(mtm_ind, col='close', N=20).loc[:, ['MTM','MTMMA']]
mtm.head()

MTM  MTMMA
date       code              
2020-09-14 000001  NaN    NaN
           000002  NaN    NaN
           000004  NaN    NaN
           000005  NaN    NaN
           000006  NaN    NaN

In [7]:
smtm = fbase.loc[fbase.ipodays > 180, :].join(mtm)
smtm.head()

open   high    low  close    volume        amount  adj  \
date       code                                                              
2020-09-14 000001  15.01  15.30  14.92  15.30  680251.0  1.027508e+09  1.0   
           000002  28.01  28.30  27.51  27.70  588402.0  1.630472e+09  1.0   
           000004  22.99  23.35  22.91  23.18   11770.0  2.722349e+07  1.0   
           000005   2.88   2.90   2.84   2.86   89083.0  2.551092e+07  1.0   
           000006   5.87   5.88   5.70   5.80  171019.0  9.886574e+07  1.0   

                   ipo_date  industry  ipodays  MTM  MTMMA  
date       code                                             
2020-09-14 000001  19910403         1    10757  NaN    NaN  
           000002  19910129        11    10821  NaN    NaN  
           000004  19910114        24    10836  NaN    NaN  
           000005  19901210        39    10871  NaN    NaN  
           000006  19920427        11    10367  NaN    NaN

In [8]:
smtm['rank'] = smtm.groupby('date')['MTM'].rank(method='dense', ascending=False)
smtm.loc[smtm['rank']<TOP_STOCKS,:].head(10)

open   high    low  close    volume        amount  adj  \
date       code                                                              
2020-10-20 000001  17.48  17.60  17.25  17.54  960071.0  1.673173e+09  1.0   
           000004  26.01  26.45  25.48  25.63   57072.0  1.479295e+08  1.0   
           000034  30.49  30.67  29.23  29.88  239448.0  7.169641e+08  1.0   
           000039   9.50  10.10   9.41  10.03  284995.0  2.817357e+08  1.0   
           000088   6.97   6.97   6.77   6.91  192342.0  1.318976e+08  1.0   
           000333  74.51  76.98  74.51  76.30  154353.0  1.175375e+09  1.0   
           000338  15.85  15.95  15.66  15.94  287346.0  4.549082e+08  1.0   
           000411  20.75  22.97  20.60  22.57  307498.0  6.804322e+08  1.0   
           000502   8.07   8.25   7.96   8.22   26106.0  2.127707e+07  1.0   
           000547  23.14  23.60  23.06  23.48  216386.0  5.053885e+08  1.0   

                   ipo_date  industry  ipodays       MTM  MTMMA   rank  
date       code                                                         
2020-10-20 000001  19910403         1    10793  0.146405    NaN  321.0  
           000004  19910114        24    10872  0.105695    NaN  486.0  
           000034  19940509        25     9661  0.236755    NaN  131.0  
           000039  19940408        26     9692  0.129505    NaN  378.0  
           000088  19970728         8     8485  0.191379    NaN  202.0  
           000333  20130918        23     2589  0.122059    NaN  415.0  
           000338  20070430         7     4922  0.121254    NaN  420.0  
           000411  19960716        12     8862  0.386364    NaN   40.0  
           000502  19921123        25    10193  0.121419    NaN  418.0  
           000547  19931130        24     9821  0.207819    NaN  174.0

In [9]:
raw = pd.read_csv("c:\\Users\\mUSicX\\Documents\\股票相关\\20200922SW.csv")
raw['c'] = raw.code.apply(lambda x: x[:6])
raw['mkt'] = raw.code.apply(lambda x: 0 if x.endswith('SZ') else 1)
raw['ind_cat1'] = raw.ind.apply(lambda x: x.split('--')[0])
raw['ind_cat2'] = raw.ind.apply(lambda x: x.split('--')[1].replace('Ⅱ', ''))
raw['ind_cat3'] = raw.ind.apply(lambda x: x.split('--')[2].replace('Ⅲ', ''))

raw['ind_cat'] = raw.apply(lambda x: x.ind_cat1+'-'+x.ind_cat2, axis=1)
raw[:10]

,code,ind,c,mkt,ind_cat1,ind_cat2,ind_cat3,ind_cat
0,000009.SZ,综合--综合Ⅱ--综合Ⅲ,000009,0,综合,综合,综合,综合-综合
1,000409.SZ,综合--综合Ⅱ--综合Ⅲ,000409,0,综合,综合,综合,综合-综合
2,000632.SZ,综合--综合Ⅱ--综合Ⅲ,000632,0,综合,综合,综合,综合-综合
3,000638.SZ,综合--综合Ⅱ--综合Ⅲ,000638,0,综合,综合,综合,综合-综合
4,000652.SZ,综合--综合Ⅱ--综合Ⅲ,000652,0,综合,综合,综合,综合-综合
5,000839.SZ,综合--综合Ⅱ--综合Ⅲ,000839,0,综合,综合,综合,综合-综合
6,002967.SZ,综合--综合Ⅱ--综合Ⅲ,002967,0,综合,综合,综合,综合-综合
7,300012.SZ,综合--综合Ⅱ--综合Ⅲ,300012,0,综合,综合,综合,综合-综合
8,300688.SZ,综合--综合Ⅱ--综合Ⅲ,300688,0,综合,综合,综合,综合-综合
9,300797.SZ,综合--综合Ⅱ--综合Ⅲ,300797,0,综合,综合,综合,综合-综合


In [10]:
industry = raw.loc[:, ['c', 'ind_cat']].set_index('c').to_dict()
indict = industry['ind_cat']

In [11]:
smtm = smtm.reset_index()
smtm['cat'] = smtm.apply(lambda x: indict[str(x.code)], axis=1)
smtm = smtm.loc[:, ['date', 'code', 'cat', 'rank']]
smtm.head(10)

,date,code,cat,rank
0,2020-09-14,000001,银行-银行,NaN
1,2020-09-14,000002,房地产-房地产开发,NaN
2,2020-09-14,000004,医药生物-化学制药,NaN
3,2020-09-14,000005,公用事业-环保工程及服务,NaN
4,2020-09-14,000006,房地产-房地产开发,NaN
5,2020-09-14,000007,商业贸易-商业物业经营,NaN
6,2020-09-14,000008,机械设备-运输设备,NaN
7,2020-09-14,000009,综合-综合,NaN
8,2020-09-14,000010,建筑装饰-园林工程,NaN
9,2020-09-14,000011,房地产-房地产开发,NaN


In [12]:
tops = smtm.loc[smtm['rank']<TOP_STOCKS,:].groupby(['date', 'cat'])['code'].count()
bases = smtm.groupby(['date', 'cat'])['code'].count()
rps = pd.merge(tops.reset_index(), bases.reset_index(), on=['date', 'cat'])
rps.rename(columns={'code_x':'high', 'code_y':'full'}, inplace=True)
rps.head(10)

,date,cat,high,full
0,2020-10-20,交通运输-港口,1,19
1,2020-10-20,交通运输-物流,7,40
2,2020-10-20,传媒-互联网传媒,6,60
3,2020-10-20,传媒-文化传媒,5,75
4,2020-10-20,传媒-营销传播,7,34
5,2020-10-20,公用事业-燃气,4,23
6,2020-10-20,公用事业-环保工程及服务,2,52
7,2020-10-20,公用事业-电力,4,74
8,2020-10-20,农林牧渔-农产品加工,2,17
9,2020-10-20,农林牧渔-动物保健,2,9


In [13]:
rps['score'] = rps.high / rps.full * rps.high
rps['rank'] = rps.groupby('date')['score'].rank(ascending=False, method='dense', na_option='bottom')
rps['cap'] = rps.apply(lambda r: '{}/{}'.format(r.high, r.full), axis=1)
rps.head(10)

,date,cat,high,full,score,rank,cap
0,2020-10-20,交通运输-港口,1,19,0.052632,67.0,1/19
1,2020-10-20,交通运输-物流,7,40,1.225000,20.0,7/40
2,2020-10-20,传媒-互联网传媒,6,60,0.600000,36.0,6/60
3,2020-10-20,传媒-文化传媒,5,75,0.333333,50.0,5/75
4,2020-10-20,传媒-营销传播,7,34,1.441176,17.0,7/34
5,2020-10-20,公用事业-燃气,4,23,0.695652,30.0,4/23
6,2020-10-20,公用事业-环保工程及服务,2,52,0.076923,64.0,2/52
7,2020-10-20,公用事业-电力,4,74,0.216216,56.0,4/74
8,2020-10-20,农林牧渔-农产品加工,2,17,0.235294,55.0,2/17
9,2020-10-20,农林牧渔-动物保健,2,9,0.444444,45.0,2/9


In [14]:
rps = rps.loc[rps.date > (datetime.today()-timedelta(days=CHECK_DAYS)).strftime('%Y-%m-%d'), :]

In [15]:
ind_pivot = rps.pivot(index='cat', columns='date', values=['score','rank', 'cap'])
ind_pivot = ind_pivot.reindex(sorted(ind_pivot.columns, reverse=True), axis=1)
ind_pivot.fillna(0, inplace=True)
ind_pivot.sort_values(by=ind_pivot.columns[0], ascending=False, inplace=True)
top_ind = ind_pivot.loc[(ind_pivot.score > 1).any(axis=1),:]
top_ind

score                                              \
date         2020-11-12 2020-11-11 2020-11-10 2020-11-09 2020-11-06   
cat                                                                   
食品饮料-饮料制造     24.025000  15.625000  13.225000   6.400000   7.225000   
化工-化学制品        8.018182   7.272727   4.950000   4.654545   5.278539   
有色金属-工业金属      6.352941   5.019608   4.411765   1.960784   0.490196   
轻工制造-家用轻工      5.633803   5.633803   4.070423   3.169014   4.070423   
汽车-汽车整车        4.545455   4.545455   4.545455   8.909091  10.227273   
机械设备-通用机械      4.283186   3.539823   6.451327   6.938053   6.877193   
家用电器-白色家电      4.083333   3.520833   3.000000   4.687500   4.687500   
汽车-汽车零部件       3.666667   4.363636   4.734848   7.757576   9.280303   
钢铁-钢铁          3.558824   4.235294   1.882353   1.058824   1.058824   
建筑材料-玻璃制造      3.125000   2.000000   3.125000   3.125000   3.125000   
化工-化学纤维        2.782609   2.782609   2.782609   1.565217   1.086957   
采掘-其他采掘        2.500000   2.500000   1.600000   1.600000   1.600000   
电子-半导体         2.283019   3.188679   4.245283   3.698113   1.528302   
电气设备-电气自动化设备   2.272727   1.454545   2.750000   2.272727   4.454545   
电子-其他电子        2.133333   1.200000   1.200000   2.700000   1.633333   
机械设备-专用设备      2.039548   2.750000   2.272727   2.750000   3.571429   
有色金属-稀有金属      1.960000   1.960000   1.000000   1.440000   1.440000   
化工-橡胶          1.894737   1.894737   0.842105   1.315789   1.315789   
电子-电子制造        1.805970   2.149254   2.149254   2.925373   2.925373   
电气设备-电源设备      1.635135   1.351351   3.905405   4.878378   4.945205   
电子-光学光电子       1.613333   1.920000   1.613333   1.920000   1.613333   
通信-通信设备        1.469388   1.469388   2.295918   2.295918   2.295918   
计算机-计算机应用      1.190476   0.640212   0.894180   1.529101   1.537234   
公用事业-燃气        1.086957   1.086957   0.391304   0.391304   0.695652   
传媒-营销传播        1.058824   1.441176   1.441176   2.941176   2.941176   
医药生物-医疗服务      1.041667   0.666667   1.500000   1.041667   1.041667   
交通运输-航运        0.900000   1.600000   1.600000   1.600000   0.900000   
交通运输-物流        0.900000   0.625000   0.625000   0.625000   0.400000   
电气设备-高低压设备     0.765625   0.562500   1.265625   1.562500   0.777778   
采掘-煤炭开采        0.657895   2.131579   0.421053   0.236842   0.236842   
电子-元件          0.609756   0.390244   0.390244   0.390244   0.097561   
纺织服装-纺织制造      0.516129   1.161290   1.580645   1.161290   1.161290   
机械设备-仪器仪表      0.470588   1.441176   2.382353   2.941176   1.882353   
汽车-其他交运设备      0.400000   0.900000   0.900000   1.600000   1.600000   
医药生物-化学制药      0.339623   0.339623   0.462264   0.150943   0.152381   
化工-塑料          0.290323   0.806452   2.064516   3.903226   3.225806   
机械设备-金属制品      0.235294   0.235294   0.235294   0.235294   0.529412   
纺织服装-服装家纺      0.173077   0.692308   0.942308   0.480769   0.692308   
医药生物-医疗器械      0.134328   0.134328   0.238806   0.373134   0.136364   
银行-银行          0.111111   0.250000   0.250000   0.111111   0.250000   
医药生物-生物制品      0.108108   0.000000   0.027027   0.027027   0.027027   
国防军工-地面兵装      0.071429   0.000000   0.000000   0.000000   0.000000   
国防军工-航空装备      0.032258   0.032258   0.032258   0.032258   0.032258   
电气设备-电机        0.000000   0.000000   1.000000   1.562500   1.562500   
国防军工-航天装备      0.000000   0.000000   0.000000   0.000000   0.071429   
非银金融-保险        0.000000   0.000000   0.000000   0.000000   0.000000   

                                                                     ...  \
date         2020-11-05 2020-11-04 2020-11-03 2020-11-02 2020-10-30  ...   
cat                                                                  ...   
食品饮料-饮料制造      6.400000   4.900000   4.225000   5.625000   5.625000  ...   
化工-化学制品        5.593607   6.593607   7.305936   7.305936   7.272727  ...   
有色金属-工业金属      0.960784   0.313725   0.176471   0.019608   0.078431  ...   
轻工制造-家用轻工      3.169014   3.169014   4.070423   1.704225   1.408451  ...   
汽车-汽车整

In [16]:
top_ind.to_excel('c:\\Users\\mUSicX\\Desktop\\block_rps_scores{}.xls'.format(TOP_STOCKS))

In [ ]:
mtest = mtm.reset_index()
mtest.loc[mtest.code=='300758',:]